# Gradient Boosted Trees with the Titanic

### Due 03/14/2016
### By Jacob Metzger

Goal: Create a best-attempt model for determining survival on the Titanic using Gradient Boosted Trees.

In [1]:
from __future__ import division #for floating division
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
%matplotlib inline
np.random.seed(314) # Set for reproducibility. 

In [2]:
# The following is clipped from the Random Forests == Awesome notebook in the course notes

# Here is a simple function to show descriptive stats on the categorical variables
def describe_categorical(X):
    """
    Just like .describe(), but returns the results for
    categorical variables only.
    """
    from IPython.display import display, HTML
    display(HTML(X[X.columns[X.dtypes == "object"]].describe().to_html()))

In [3]:
#Another function taken from the Random Forests == Awesome notebook from the course page

# Look at all the columns in the dataset
def printall(X, max_rows=10):
    from IPython.display import display, HTML
    display(HTML(X.to_html(max_rows=max_rows)))

### Data processing functions/options

In [4]:
#Handle PassengerId
def handlePassengerId(X):
    return X #don't do anything

def dropPassengerId(X):
    return X.drop(["PassengerId"], axis=1) #Drop the PassengerId at the onset -- any value this variable has is elusive

In [5]:
#Handle Ticket
def handleTicket(X): #Handles ticket by splitting the ticket number and ticket prefix, providing the number and a boolean variable instead
    ticketsSplit = [_.split(" ") for _ in X.Ticket] #split the ticket codes into prefix and number
    ticketNumbers = [int(_[-1]) if _[-1]!='LINE' else 0 for _ in ticketsSplit] 
    ticketNumbersTransform = np.log(np.array(ticketNumbers)+1)
    hasTicketPrefix = [1 if _.__len__()>1 else 0 for _ in ticketsSplit] 
    X = pd.concat([X, pd.DataFrame(ticketNumbersTransform, columns=['TicketNumTrans']), pd.DataFrame(hasTicketPrefix, columns=["HasTicketPrefix"])], axis=1)
    X=X.drop(["Ticket"], axis=1)
    return X

def dropTicket(X):
    return X.drop(["Ticket"], axis=1)

In [6]:
#Handle Name
def handleName(X): #Handles Name by taking the prefix, correcting for some odd values, and providing dummy variables
    prefixes = [name.split(",")[1].split(" ")[1] for name in X.Name]
    for i, item in enumerate(prefixes):
        if prefixes[i] in ["Mr.", "Miss.", "Mrs.", "Master."]:
            #do nothing
            pass
        elif prefixes[i] in ["Mlle."]:
            prefixes[i]="Miss."
        elif prefixes[i] in ["Ms.", "Mme."]:
            prefixes[i]="Mrs."
        elif prefixes[i] in ["Jonkheer.", "Sir.", "Don."]:
            prefixes[i] = "Nobility"
        elif prefixes[i] in ["Lady.", "the"]: #"the" is for the countess
            prefixes[i] = "Nobility"
        elif prefixes[i] in ["Col.", "Major.", "Capt."]:
            prefixes[i]="CrewMember"
        elif prefixes[i] in ["Dr.", "Rev."]:
            prefixes[i]="CrewMember"
        else:
            print item # There should be no output if all prefixes are handled correctly!
    X = pd.concat([X, pd.get_dummies(prefixes)], axis=1)
    X = X.drop(["Name"], axis=1)
    return X

def dropName(X):
    return X.drop(["Name"], axis=1)

In [7]:
#Handle SibSp
def handleSibSp(X): #Handles SibSp with a mathematical transform
    X = pd.concat([X, pd.DataFrame([np.exp(-1/(x+1)**2) for x in X.SibSp], columns=["SibSpTransform"])], axis=1)
    X = X.drop(['SibSp'], axis=1)
    return X

def dropSibSp(X):
    return X.drop(["SibSp"], axis=1)

In [8]:
#Handle Parch
def handleParch(X): #Handles Parch with a mathematical transform
    X = pd.concat([X, pd.DataFrame([np.exp(-1/(x+1)**2) for x in X.Parch], columns=["ParchTransform"])], axis=1)
    X = X.drop(['Parch'], axis=1)
    return X

def dropParch(X):
    return X.drop(['Parch'], axis=1)

In [9]:
def haveFamily((x,y)):
    if x>0 or y>0:
        return 1
    else:
        return 0

def addFamily(X):
    X=pd.concat([X, pd.DataFrame(map(haveFamily, zip(X.Parch, X.SibSp)), columns=["Family"])], axis=1)
    return X

In [10]:
#Handle Cabin
def handleCabin(X): #Handles cabin by providing both a cabin count (imputing a mean, when necessary) and the cabin letter
    cabinSplit = [str(_).split(" ") for _ in X.Cabin] #tokenize the cabins
    cabinsFinalSplit = [[cabin if cabin=="nan" else [cabin[0],cabin[1:]] for cabin in cabins] for cabins in cabinSplit] 

    numCabins = [0 if _[0]=="nan" else _.__len__() for _ in cabinsFinalSplit ]
    X = pd.concat([X, pd.DataFrame(numCabins, columns=["NumCabins"])], axis=1)
    X.NumCabins=X.NumCabins.replace(0, X.NumCabins.mean())

    cabinLetters = ["None" if cabin[0]=="nan" else cabin[0][0] for cabin in cabinsFinalSplit]
    X=pd.concat([X, pd.get_dummies(pd.DataFrame(cabinLetters, columns=["CabinLetters"]))], axis=1)
    X=X.drop(["Cabin"], axis=1)
    
    return X

def dropCabin(X):
    return X.drop(["Cabin"], axis=1)

In [11]:
#Handle PClass
def handlePclass(X): #Provides dummy variables for Pclass
    X = pd.concat([X, pd.get_dummies(X.Pclass)], axis=1)
    X = X.drop(["Pclass"], axis=1)
    return X

def dropPclass(X):
    return X.drop(["Pclass"], axis=1)

In [12]:
#Handle Sex
def handleSex(X): #Provides dummy variables for Sex
    X.Sex = pd.get_dummies(X.Sex).female #Just replace the Sex column altogether
    return X

def dropSex(X):
    return X.drop(["Sex"], axis=1)

In [13]:
#Handle Embarked
def handleEmbarked(X): #Provides dummy variables for Embarked after imputing missing values with the mode
    X.Embarked = X.Embarked.fillna(X.Embarked.mode())
    X = pd.concat([X, pd.get_dummies(X.Embarked)], axis=1)
    X = X.drop(["Embarked"], axis=1)
    return X

def dropEmbarked(X):
    return X.drop(["Embarked"], axis=1)

In [14]:
#Handle Fare
def handleFareBasic(X): #Handles Fare by imputing the mean into N/A and 0 values
    originalFare = X.Fare
    X.Fare = X.Fare.fillna(originalFare.mean())
    X.Fare = X.Fare.replace(0, originalFare.mean()) #It seems odd to have zero fares... Probably outliers anyway, so impute them.
    return X

def handleFareAdvanced(X):
    X_copy=X.copy()
    fareDataset = meanAge(X_copy) 
    fareDataset.dropna(axis=0)
    
    fareTrainingSet = fareDataset[fareDataset.Fare!=0]
    #print fareTrainingSet.Fare.min()
    fareTarget = fareTrainingSet.pop("Fare")
    #print fareTarget.shape
    #print fareTrainingSet.shape
    
    from sklearn.ensemble import GradientBoostingRegressor
    fareRegressor = GradientBoostingRegressor(n_estimators=100)
    fareRegressor.fit(fareTrainingSet, fareTarget)
    
    counter=0
    for row in xrange(0,len(X_copy.Fare)):
        if X.Fare[row]==0:
            X.Fare[row]=fareRegressor.predict(X_copy.iloc[row].drop("Fare").reshape(1,-1))
            #print X.iloc[row]
            counter+=1
    return X
    
    
def binFare(X): #Handles Fare by binning it into a set of bins
    numFareBins = 10
    fareBins = pd.cut(X.Fare, numFareBins, labels=[str(_)+"Fare" for _ in range(0, numFareBins, 1)])
    X = pd.concat([X, pd.get_dummies(fareBins)], axis=1)
    X = X.drop(["Fare"], axis=1)
    return X
    
def logFare(X): #Handles Fare with a log transform
    X.Fare = np.log(X.Fare+1)
    return X
    
def dropFare(X):
    return X.drop(["Fare"], axis=1)

In [15]:
#Handle Age
def handleAge(X): #Fills missing values of Age with the predictions of a Random Forest regression on the other variables
    ageDataset=X.dropna(axis=0)
    ageTarget = pd.DataFrame(ageDataset.Age)
    ageDataset = ageDataset.drop(["Age"], axis=1)

    import numpy
    datasetMissingAgeVals = X[numpy.isnan(X.Age)]
    datasetMissingAgeVals = datasetMissingAgeVals.drop(["Age"], axis=1)

    #from sklearn.ensemble import RandomForestRegressor
    #ageClassifier= RandomForestRegressor(n_estimators=2000, random_state=314)
    from sklearn.ensemble import GradientBoostingRegressor
    ageClassifier = GradientBoostingRegressor()
    ageClassifier.fit(ageDataset, ageTarget.astype(int))
    guessedAges = ageClassifier.predict(datasetMissingAgeVals)

    counter=0
    for row in xrange(0,len(X.Age)):
        if numpy.isnan(X.Age[row]):
            X.Age[row]=guessedAges[counter]
            counter+=1
    return X

def dropAge(X):
    return X.drop(["Age"], axis=1)

def binAge(X): #Handles Age by first getting the values filled then binning into categories Child, Adult, and Elderly
    X = handleAge(X)
    from collections import defaultdict
    triBins = defaultdict(str)
    for i in xrange(len(X.Age)):
        if X.Age.iloc[i]<16:
            triBins[i]="Child"
        elif X.Age.iloc[i]<55:
            triBins[i]="Adult"
        else:
            triBins[i]="Elderly"
    X = pd.concat([X, pd.get_dummies(triBins)], axis=1)
    X = X.drop(["Child"], axis=1)
    X = X.drop(["Age"], axis=1)
    return X

def meanAge(X): #Imputes missing values of Age with the mean
    X.Age = X.Age.fillna(X.Age.mean())
    return X

In [16]:
#Here I considered trying some arithmetic transformations, but it didn't look like they ended up being useful.

#Create some new features with basic arithmetic operations
def deriveNewColumns(X):
    newdataset = X.copy()
    for x1 in xrange(0, len(X.columns)):
        for x2 in xrange(x1+1, len(X.columns)):
            col1name = X.columns[x1]
            col2name = X.columns[x2]
            #print col1name, col2name
            newAddDF = pd.DataFrame(X[col1name]+X[col2name], columns = [str(col1name)+"+"+str(col2name)])
            newSubDF = pd.DataFrame(X[col1name]-X[col2name], columns = [str(col1name)+"-"+str(col2name)])
            newMulDF = pd.DataFrame(X[col1name]*X[col2name], columns = [str(col1name)+"*"+str(col2name)])
            #newDivDF -- don't want to deal with div by zero 
            newdataset = pd.concat([newdataset, newAddDF, newSubDF, newMulDF], axis=1)
            #newdataset = pd.concat([newdataset, newAddDF], axis=1)
    return newdataset.copy()

## Begin Work

In [17]:
titanicDataset = pd.read_csv("train.csv")
X = titanicDataset.drop(["Survived"], axis=1)
y = titanicDataset.Survived

In [18]:
X.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [19]:
describe_categorical(X) 

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Graham, Mr. George Edward",male,CA. 2343,C23 C25 C27,S
freq,1,577,7,4,644


In [20]:
#Process data here
#The variables presented to the model are altered by the processing done here.

#X = dropPassengerId(X)

#X = addFamily(X)

X = handlePclass(X)
#X = dropPclass(X)

X = handleName(X)
#X = dropName(X)

X = handleSex(X)
#X = dropSex(X)

X = handleSibSp(X)
#X = dropSibSp(X)

X = handleParch(X)
#X = dropParch(X)

X = handleTicket(X)
#X = dropTicket(X)

X = handleCabin(X)
#X = dropCabin(X)

X = handleEmbarked(X)
#X = dropEmbarked(X)

X = handleFareBasic(X)
#X = handleFareAdvanced(X) #This doesn't seem to help. Modeled on how we're handling age
#X = binFare(X)
#X = logFare(X)
#X = dropFare(X)

X = handleAge(X)  # Do this last because of the way we're imputing Age
#X = binAge(X)
#X = meanAge(X)
#X = dropAge(X)

#X = deriveNewColumns(X) # This just appears to add variance

C:\Users\Izzy\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Izzy\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
X.describe()

,PassengerId,Sex,Age,Fare,1,2,3,CrewMember,Master.,Miss.,...,CabinLetters_C,CabinLetters_D,CabinLetters_E,CabinLetters_F,CabinLetters_G,CabinLetters_None,CabinLetters_T,C,Q,S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.352413,29.654892,32.746366,0.242424,0.206510,0.551066,0.020202,0.044893,0.206510,...,0.066218,0.037037,0.035915,0.014590,0.004489,0.771044,0.001122,0.188552,0.086420,0.722783
std,257.353842,0.477990,13.588875,49.514272,0.428790,0.405028,0.497665,0.140770,0.207186,0.405028,...,0.248802,0.188959,0.186182,0.119973,0.066890,0.420397,0.033501,0.391372,0.281141,0.447876
min,1.000000,0.000000,0.420000,4.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,21.000000,7.925000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,28.613707,15.100000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,37.000000,32.204208,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,80.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
#Scale the set
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X))

### Gridsearch on Hyperparameters for Gradient Boosted Trees

In [23]:
#The following commented code is adapted directly from lecture entitled Random Forests == Awesome
#Remove # as necessary to search different combinations of parameters

#from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.grid_search import GridSearchCV
##n_estimator_options = [200, 500, 1000, 1500, 2000]
##n_estimator_options = [100,200,300,400,500]
##n_estimator_options = [80,90,100,110,120,130,140, 200]
#n_estimator_options=[100]
##max_features_options = ["auto", None, "sqrt", "log2", 0.9, 0.2]
#max_features_options = ["log2"]
##min_samples_leaf_options = [1, 2, 3, 4, 5]
##min_samples_leaf_options = [3,4,5,6,7,8]
#min_samples_leaf_options=[8]
##min_samples_split_options = [1,2,3,4,5]
#min_samples_split_options = [1]
##max_depth_options = [1,2,3,4,5,6,7,8,9,10]
#max_depth_options = [4]
#max_leaf_nodes_options = [2,3,4,5, None]
#min_weight_fraction_leaf_options = [0.0,0.1,0.2,0.3,0.4]

#gbModel = GradientBoostingRegressor()
#estimator = GridSearchCV(gbModel, dict(
#        n_estimators=n_estimator_options,
#        max_features=max_features_options,
#        min_samples_leaf=min_samples_leaf_options,
#        min_samples_split=min_samples_split_options,
#        max_depth=max_depth_options,
#        max_leaf_nodes = max_leaf_nodes_options,
#        min_weight_fraction_leaf = min_weight_fraction_leaf_options
#    ), cv=3, n_jobs=-2, scoring="roc_auc")

In [24]:
#estimator.fit(X, y)

In [25]:
#estimator.best_estimator_

In [26]:
#gbModel = estimator.best_estimator_

### Final Model

In [27]:
# Make train and test datasets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

from sklearn.ensemble import GradientBoostingRegressor
#These hyperparameters were picked by the above gridsearch
gbModel = GradientBoostingRegressor(n_estimators=100, 
                                    max_depth=4,
                                    max_features="log2", 
                                    max_leaf_nodes=None,
                                    min_samples_leaf=8, 
                                    min_samples_split=1, 
                                    min_weight_fraction_leaf=0.0,
                                    random_state=42)
gbModel.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=4, max_features='log2', max_leaf_nodes=None,
             min_samples_leaf=8, min_samples_split=1,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=42, subsample=1.0, verbose=0,
             warm_start=False)

In [28]:
from sklearn.cross_validation import cross_val_score
rocScores = cross_val_score(gbModel, X, y, cv=10, scoring="roc_auc", n_jobs=-2)
mseScores = cross_val_score(gbModel, X, y, cv=10, scoring="mean_squared_error", n_jobs=-2) #These return negative by convention per docs.
mseScores*=-1
print rocScores
print mseScores

[ 0.86475616  0.83876812  0.82994652  0.87045455  0.91346154  0.90186404
  0.87282051  0.9025974   0.95028249  0.875     ]
[ 0.15021344  0.12098253  0.15875597  0.12391353  0.10942583  0.12129104
  0.1341797   0.12474921  0.09178329  0.12207822]


In [29]:
from scipy.stats import sem
print "ROC AUC: ",rocScores.mean()," +- ", 2.262*sem(rocScores, ddof=0) #Mean ROC AUC and 95% ci
print "MSE:     ",mseScores.mean()," +- ", 2.262*sem(mseScores, ddof=0) #Mean MSE and 95% ci

ROC AUC:  0.881995131918  +-  0.0243767939732
MSE:      0.125737275878  +-  0.0128747261386
